In [ ]:
import pandas as pd

In [ ]:
!pip install sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 4.8 MB/s 
     |████████████████████████████████| 4.7 MB 52.5 MB/s 
     |████████████████████████████████| 1.3 MB 46.5 MB/s 
     |████████████████████████████████| 101 kB 11.2 MB/s 
     |████████████████████████████████| 596 kB 68.4 MB/s 
     |████████████████████████████████| 6.6 MB 57.3 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=550abc9aeebac73fbd4367e899d1886c33e8e024806d19a672682ae756f50664
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from sklearn.metrics import confusion_matrix,f1_score,accuracy_score
from sentence_transformers import SentenceTransformer, util
import numpy as np
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader

In [ ]:
df = pd.read_csv('/content/evaluation.csv')

In [ ]:
train_df = pd.read_csv('/content/semantic_train.csv')

In [ ]:
def pretrained_model(model_name,df):
  model = SentenceTransformer(model_name)
  similarity= []
  for i in range(len(df)):
      sentence1 = df.iloc[i][0]
      sentence2 = df.iloc[i][1]
      #print(sentence1,sentence2)
      embedding1 = model.encode(sentence1, convert_to_tensor=True)
      embedding2 = model.encode(sentence2, convert_to_tensor=True)
      cosine_scores = util.pytorch_cos_sim(embedding1, embedding2)
      similarity.append(float(cosine_scores[0][0]))
  return similarity


def get_scores(similarity,real_values,threshold = 0.5):
  train_result = [1 if i > threshold else 0 for i in similarity]
  print('confusion_matrix',confusion_matrix(real_values,train_result))
  print('f1_score',f1_score(real_values,train_result))
  print('accuracy_score',accuracy_score(real_values,train_result))


In [ ]:
similarity= pretrained_model('nghuyong/ernie-2.0-en',df)
get_scores(similarity,df['label'])

Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/nghuyong_ernie-2.0-en were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


confusion_matrix [[   0 5999]
 [   0 3001]]
f1_score 0.5001249895842013
accuracy_score 0.33344444444444443


In [ ]:
#removing stop words and lemmatization 
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
similarity_after_textprocess = []
lem = WordNetLemmatizer()
for i in range(len(df)):
    sentence1 = word_tokenize(df.iloc[i][0])
    sentence2 = word_tokenize(df.iloc[i][1])
    sentence1 = [lem.lemmatize(word) for word in sentence1 if word not in stopwords.words('english')]
    sentence2 = [lem.lemmatize(word) for word in sentence2 if word not in stopwords.words('english')]
    sentence1 = ' '.join(sentence1)
    sentence2 = ' '.join(sentence2)
    embedding1 = model.encode(sentence1, convert_to_tensor=True)
    embedding2 = model.encode(sentence2, convert_to_tensor=True)
    cosine_scores = util.pytorch_cos_sim(embedding1, embedding2)
    similarity_after_textprocess.append(float(cosine_scores[0][0]))

In [ ]:
train_result = [1 if i > 0.5 else 0 for i in similarity_after_textprocess]
print('confusion_matrix',confusion_matrix(df['label'],train_result))
print('f1_score',f1_score(df['label'],train_result))
print('accuracy_score',accuracy_score(df['label'],train_result))

confusion_matrix [[4500 1499]
 [1061 1940]]
f1_score 0.6024844720496895
accuracy_score 0.7155555555555555


In [ ]:
model = SentenceTransformer('microsoft/mpnet-base')

Downloading:   0%|          | 0.00/345 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/493 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/532M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/472k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/microsoft_mpnet-base were not used when initializing MPNetModel: ['lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing MPNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MPNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of MPNetModel were not initialized from the model checkpoint at /root/.cache/torch/sentence_transformers/microsoft_mpnet-base and are newly initialized: ['mpnet.pooler.dense.weight', 'mpnet.pooler.dense.bias']
You sho

In [ ]:
model = SentenceTransformer('bert-base-nli-mean-tokens')
similarity= []
for i in range(100):
    sentence1 = df.iloc[i][0]
    sentence2 = df.iloc[i][1]
    #print(sentence1,sentence2)
    embedding1 = model.encode(sentence1, convert_to_tensor=True)
    embedding2 = model.encode(sentence2, convert_to_tensor=True)
    cosine_scores = util.pytorch_cos_sim(embedding1, embedding2)
    similarity.append(float(cosine_scores[0][0]))

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.95k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [ ]:
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
import math
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
import numpy as np
from sklearn.metrics import roc_auc_score

In [ ]:
def train_model(model_name,df):
  X_train, X_test, y_train, y_test = train_test_split(df[['text','reason']], df['label'], test_size=0.4, random_state=42)
  X_dev, X_test, y_dev, y_test = train_test_split(X_test , y_test, test_size=0.5, random_state=42)
  X_train = np.array(X_train)
  X_test = np.array(X_test)
  y_train = np.array(y_train)
  y_test = np.array(y_test)
  X_dev = np.array(X_dev)
  y_dev = np.array(y_dev)

  train_batch_size = 16
  num_epochs = 4
  model = SentenceTransformer(model_name)
  train_samples = []
  dev_samples = []
  test_samples = []
  for i in range(len(X_train)):
      inp_example = InputExample(texts=[X_train[i][0], X_train[i][1]], label=float(y_train[i]))
      train_samples.append(inp_example)
      
  for i in range(len(X_dev)):
      inp_example = InputExample(texts=[X_dev[i][0], X_dev[i][1]], label=float(y_dev[i]))
      dev_samples.append(inp_example)
      
      inp_example = InputExample(texts=[X_test[i][0], X_test[i][1]], label=float(y_test[i]))
      test_samples.append(inp_example)
  
  evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_samples, name='sts-dev')
  train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=train_batch_size)

  train_loss = losses.CosineSimilarityLoss(model)
  warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1)
  model.fit(train_objectives=[(train_dataloader, train_loss)],
            evaluator=evaluator,
            epochs=num_epochs,
            evaluation_steps=1000,
            warmup_steps=warmup_steps,
            output_path='/content/model_output')
  return model
    

def trained_model_scores(model,df,threshold = 0.5):
  similarity= []
  for i in range(len(df)):
      sentence1 = df.iloc[i][0]
      sentence2 = df.iloc[i][1]
      #print(sentence1,sentence2)
      embedding1 = model.encode(sentence1, convert_to_tensor=True)
      embedding2 = model.encode(sentence2, convert_to_tensor=True)
      cosine_scores = util.pytorch_cos_sim(embedding1, embedding2)
      similarity.append(float(cosine_scores[0][0]))

  train_result = [1 if i > threshold else 0 for i in similarity]
  print('confusion_matrix',confusion_matrix(df['label'],train_result))
  print('f1_score',f1_score(df['label'],train_result))
  print('roc_auc_score',roc_auc_score(df['label'],train_result))
  print('accuracy_score',accuracy_score(df['label'],train_result))



In [ ]:
model = train_model('nli-distilroberta-base-v2',df)

Downloading:   0%|          | 0.00/736 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.71k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/679 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/338 [00:00<?, ?it/s]

Iteration:   0%|          | 0/338 [00:00<?, ?it/s]

Iteration:   0%|          | 0/338 [00:00<?, ?it/s]

Iteration:   0%|          | 0/338 [00:00<?, ?it/s]

In [ ]:
trained_model_scores(model,train_df)

confusion_matrix [[2061]]
f1_score 1.0
accuracy_score 1.0


In [ ]:
trained_model_scores(model,df)

confusion_matrix [[5687  312]
 [ 268 2733]]
f1_score 0.9040688058220311
roc_auc_score 0.9293438832052372
accuracy_score 0.9355555555555556


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df[['text','reason']], df['label'], test_size=0.4, random_state=42)
X_dev, X_test, y_dev, y_test = train_test_split(X_test , y_test, test_size=0.5, random_state=42)
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)
X_dev = np.array(X_dev)
y_dev = np.array(y_dev)

In [ ]:
model_name = 'nli-distilroberta-base-v2'
train_batch_size = 16
num_epochs = 4
model = SentenceTransformer(model_name)
train_samples = []
dev_samples = []
test_samples = []
for i in range(len(X_train)):
    inp_example = InputExample(texts=[X_train[i][0], X_train[i][1]], label=float(y_train[i]))
    train_samples.append(inp_example)

for i in range(len(X_dev)):
    inp_example = InputExample(texts=[X_dev[i][0], X_dev[i][1]], label=float(y_dev[i]))
    dev_samples.append(inp_example)
    
    inp_example = InputExample(texts=[X_test[i][0], X_test[i][1]], label=float(y_test[i]))
    test_samples.append(inp_example)




Downloading:   0%|          | 0.00/736 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.71k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/679 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [ ]:
evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_samples, name='sts-dev')
train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=train_batch_size)

In [ ]:
train_loss = losses.CosineSimilarityLoss(model)
warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1)

In [ ]:
model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=evaluator,
          epochs=num_epochs,
          evaluation_steps=1000,
          warmup_steps=warmup_steps,
          output_path='/content/model_output')

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/338 [00:00<?, ?it/s]

Iteration:   0%|          | 0/338 [00:00<?, ?it/s]

Iteration:   0%|          | 0/338 [00:00<?, ?it/s]

Iteration:   0%|          | 0/338 [00:00<?, ?it/s]

In [ ]:
from sentence_transformers import util

In [ ]:
similarity = []
for i in range(len(train_df)):
  sentence1 = train_df.iloc[i][0]
  sentence2 = train_df.iloc[i][1]
  #print(sentence1,sentence2)
  embedding1 = model.encode(sentence1, convert_to_tensor=True)
  embedding2 = model.encode(sentence2, convert_to_tensor=True)
  cosine_scores = util.pytorch_cos_sim(embedding1, embedding2)
  similarity.append(float(cosine_scores[0][0]))

In [ ]:
train_result = [1 if i > 0.35 else 0 for i in similarity]
print('confusion_matrix',confusion_matrix(train_df['label'],train_result))
print('f1_score',f1_score(train_df['label'],train_result))
print('accuracy_score',accuracy_score(train_df['label'],train_result))

confusion_matrix [[   0    0]
 [ 680 1381]]
f1_score 0.8024404416037187
accuracy_score 0.6700630761766133


In [ ]:
train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=1)
train_loss = losses.CosineSimilarityLoss(model)
model.fit(train_objectives=[(train_dataloader, train_loss)],evaluator=evaluator, epochs=1, warmup_steps=100)

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/9000 [00:00<?, ?it/s]

RuntimeError: ignored

In [ ]:
df

,text,reason,label
0,the app is crashing when i play a vedio,app crashes during playback,1
1,but i want to connect it to the tv from one de...,want compatibility with more smart televisions,0
2,very helpful when and home working remotley,good app for work,0
3,this zoom so called and missed call and mobile...,receiving incorrect phone number message,0
4,one of my favorite apps,good for spending time,0
...,...,...,...
8995,very good streaming platform has a lot of content,app has very good content,1
8996,so paying for an app that is not fit for purpose.,unable to use app,0
8997,"it does not work well for me, it loads, but if...",app keeps loading indefinitely,0
8998,i can not watch the new disney shows.,unable to load the episodes list,0


In [ ]:
similarity= []
for i in range(len(df)):
    sentence1 = df.iloc[i][0]
    sentence2 = df.iloc[i][1]
    #print(sentence1,sentence2)
    embedding1 = model.encode(sentence1, convert_to_tensor=True)
    embedding2 = model.encode(sentence2, convert_to_tensor=True)
    cosine_scores = util.pytorch_cos_sim(embedding1, embedding2)
    similarity.append(float(cosine_scores[0][0]))

In [ ]:
train_result = [1 if i > 0.5 else 0 for i in similarity]
print('confusion_matrix',confusion_matrix(df['label'],train_result))
print('f1_score',f1_score(df['label'],train_result))
print('accuracy_score',accuracy_score(df['label'],train_result))

confusion_matrix [[   0 5999]
 [   0 3001]]
f1_score 0.5001249895842013
accuracy_score 0.33344444444444443


In [ ]:
embedding1 = model.encode('i hate this app', convert_to_tensor=True)
embedding2 = model.encode('good app', convert_to_tensor=True)
cosine_scores = util.pytorch_cos_sim(embedding1, embedding2)

In [ ]:
cosine_scores

tensor([[1.0000]], device='cuda:0')